<a href="https://colab.research.google.com/github/41371115h-dot/114-class-thursday2-4/blob/main/%E7%A8%8B%E5%BC%8F%E8%AA%9E%E8%A8%80_%E7%AC%AC%E4%B8%80%E5%A0%82%E8%AA%B2_%E6%97%A5%E5%B8%B8%E6%94%B6%E6%94%AF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **程式語言第一堂 日常收支計算表-->日常收支結合註冊與登入的系統**

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gradio as gr
import gspread
from google.auth import default
import pandas as pd
from datetime import datetime
import ipywidgets as widgets
from IPython.display import display, clear_output
from gspread_dataframe import set_with_dataframe

# Google Sheet 連線

creds, _ = default()
gc = gspread.authorize(creds)
gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/1hShmWcTWHznIRDzpRCJMMo7TctVxpKRL2v_Dj4OY0Go/edit?usp=sharing')

def get_or_create_worksheet(gsheets, title, rows=100, cols=10, headers=None):
    try:
        ws = gsheets.worksheet(title)
    except:
        ws = gsheets.add_worksheet(title=title, rows=rows, cols=cols)
        if headers:
            ws.update([headers])
    return ws

user_sheet = get_or_create_worksheet(gsheets, 'users', headers=['帳號','密碼'])
record_sheet = get_or_create_worksheet(gsheets, 'records', headers=['日期','帳號','品項','金額','需求','CP值','分類'])

# 載入與存檔

def load_users():
    data = user_sheet.get_all_values()
    return pd.DataFrame(data[1:], columns=data[0]) if len(data) > 1 else pd.DataFrame(columns=['帳號','密碼'])

def load_records():
    data = record_sheet.get_all_values()
    return pd.DataFrame(data[1:], columns=data[0]) if len(data) > 1 else pd.DataFrame(columns=['日期','帳號','品項','金額','需求','CP值','分類'])

def save_users(df):
    user_sheet.clear()
    user_sheet.update([df.columns.values.tolist()] + df.values.tolist())

def save_records(df):
    record_sheet.clear()
    set_with_dataframe(record_sheet, df)

user_df = load_users()
current_user = None

title_label = widgets.HTML("<h2 style='color:#4B0082;'>💜 歡迎使用記帳系統 💜</h2>")

register_btn = widgets.Button(description="註冊", button_style='success', layout=widgets.Layout(width='120px'))
login_btn = widgets.Button(description="登入", button_style='info', layout=widgets.Layout(width='120px'))
message_out = widgets.Output()

username_text = widgets.Text(description="帳號:", layout=widgets.Layout(width='300px'))
password_text = widgets.Password(description="密碼:", layout=widgets.Layout(width='300px'))
submit_btn = widgets.Button(description="提交", button_style='primary', layout=widgets.Layout(width='100px'))

form_box = widgets.VBox([])

def show_message(msg, color='red'):
    with message_out:
        clear_output()
        display(widgets.HTML(f"<p style='color:{color}; font-weight:bold'>{msg}</p>"))

# 註冊 / 登入流程

def on_register_clicked(b):
    form_box.children = [widgets.HTML("<h3 style='color:#32CD32'>註冊帳號</h3>"), username_text, password_text, submit_btn]
    submit_btn.on_click(register_user)

def register_user(b):
    global user_df
    user = username_text.value.strip()
    pwd = password_text.value.strip()
    if user in user_df['帳號'].values:
        show_message("⚠️ 帳號已存在！")
    elif user == "" or pwd == "":
        show_message("⚠️ 帳號與密碼不可為空！")
    else:
        new_row = pd.DataFrame([[user, pwd]], columns=['帳號','密碼'])
        user_df = pd.concat([user_df, new_row], ignore_index=True)
        save_users(user_df)
        show_message("✅ 註冊成功！請登入", color='green')
        form_box.children = []

def on_login_clicked(b):
    form_box.children = [widgets.HTML("<h3 style='color:#1E90FF'>登入帳號</h3>"), username_text, password_text, submit_btn]
    submit_btn.on_click(login_user)

def login_user(b):
    global current_user
    user = username_text.value.strip()
    pwd = password_text.value.strip()
    if user in user_df['帳號'].values:
        stored_pwd = user_df.loc[user_df['帳號']==user, '密碼'].values[0]
        if pwd == stored_pwd:
            current_user = user
            show_message(f"✅ {user} 登入成功！", color='green')
            form_box.children = []
            show_record_form()
        else:
            show_message("❌ 密碼錯誤！")
    else:
        show_message("❌ 查無此帳號！")

# 記帳表單

categories = ['1.主餐','2.甜點','3.飲料','4.交通','5.醫療','6.繳費','7.娛樂','8.購物','9.日用品','10.其他']

def show_record_form():
    global record_df
    record_df = load_records()

    date_picker = widgets.DatePicker(description='日期', value=datetime.today())
    item_text = widgets.Text(description='品項')
    amount_text = widgets.FloatText(description='金額', value=0)
    need_dropdown = widgets.Dropdown(options=['需要','想要'], description='需求')
    cp_slider = widgets.IntSlider(value=50, min=1, max=100, description='CP值')
    category_dropdown = widgets.Dropdown(options=categories, description='分類')
    add_btn = widgets.Button(description="新增支出", button_style='success')

    output_table = widgets.Output()
    today_label = widgets.HTML("")

    # 刷新表格
    def refresh_table():
        user_records = record_df[record_df['帳號']==current_user].copy()
        user_records['金額'] = pd.to_numeric(user_records['金額'], errors='coerce').fillna(0)
        total = user_records['金額'].sum()
        today = datetime.today().strftime("%Y-%m-%d")
        today_sum = user_records[user_records['日期']==today]['金額'].sum()
        today_label.value = f"<h4 style='color:#FF4500'>今日累計支出: {today_sum}</h4>"

        with output_table:
            clear_output()
            if user_records.empty:
                display(widgets.HTML("<p>目前沒有支出紀錄</p>"))
            else:
                # 建立表格
                rows = []
                header = ["日期","品項","金額","需求","CP值","分類","操作"]
                rows.append(widgets.HBox([widgets.Label(h, layout=widgets.Layout(width="120px")) for h in header]))
                for i, row in user_records.iterrows():
                    btn = widgets.Button(description="刪除", button_style='danger', layout=widgets.Layout(width='60px'))
                    def on_delete(b, idx=i):
                        global record_df
                        record_df = record_df.drop(record_df[(record_df['帳號']==current_user) & (record_df.index==idx)].index)
                        record_df.reset_index(drop=True, inplace=True)
                        save_records(record_df)
                        refresh_table()
                    btn.on_click(on_delete)
                    row_widgets = [
                        widgets.Label(str(row["日期"]), layout=widgets.Layout(width="120px")),
                        widgets.Label(str(row["品項"]), layout=widgets.Layout(width="120px")),
                        widgets.Label(str(row["金額"]), layout=widgets.Layout(width="120px")),
                        widgets.Label(str(row["需求"]), layout=widgets.Layout(width="120px")),
                        widgets.Label(str(row["CP值"]), layout=widgets.Layout(width="120px")),
                        widgets.Label(str(row["分類"]), layout=widgets.Layout(width="120px")),
                        btn
                    ]
                    rows.append(widgets.HBox(row_widgets))
                # 總計列
                total_row = widgets.HBox([
                    widgets.Label("總計", layout=widgets.Layout(width="240px")),
                    widgets.Label(str(total), layout=widgets.Layout(width="120px"))
                ])
                rows.append(total_row)
                display(widgets.VBox(rows))

    # 新增支出
    def add_record(b):
        global record_df
        new_row = {"日期": date_picker.value.strftime("%Y-%m-%d"),
                   "帳號": current_user, "品項": item_text.value,
                   "金額": amount_text.value, "需求": need_dropdown.value,
                   "CP值": cp_slider.value, "分類": category_dropdown.value}
        record_df = pd.concat([record_df, pd.DataFrame([new_row])], ignore_index=True)
        save_records(record_df)
        item_text.value = ''
        amount_text.value = 0
        need_dropdown.value = '需要'
        cp_slider.value = 50
        category_dropdown.value = categories[0]
        refresh_table()

    add_btn.on_click(add_record)

    record_form = widgets.VBox([
        widgets.HTML(f"<h3 style='color:#4B0082'>{current_user} 的記帳表單</h3>"),
        today_label,
        date_picker, item_text, amount_text, need_dropdown, cp_slider, category_dropdown, add_btn,
        output_table
    ])
    display(record_form)
    refresh_table()

# 綁定主按鈕

register_btn.on_click(on_register_clicked)
login_btn.on_click(on_login_clicked)

# 顯示主介面

display(title_label)
display(widgets.HBox([register_btn, login_btn], layout=widgets.Layout(justify_content="flex-start")))
display(message_out)
display(form_box)

import gradio as gr

# ============= Gradio 介面 =============

def gr_register(user, pwd):
    global user_df
    if not user or not pwd:
        return "⚠️ 帳號與密碼不可為空！"
    if user in user_df['帳號'].values:
        return "⚠️ 帳號已存在！"
    user_df = pd.concat([user_df, pd.DataFrame([[user, pwd]], columns=['帳號','密碼'])], ignore_index=True)
    save_users(user_df)
    return "✅ 註冊成功！請登入"

def gr_login(user, pwd):
    global current_user
    if user not in user_df['帳號'].values:
        return "❌ 查無此帳號！"
    stored_pwd = user_df.loc[user_df['帳號']==user, '密碼'].values[0]
    if pwd != stored_pwd:
        return "❌ 密碼錯誤！"
    current_user = user
    return f"✅ {user} 登入成功！"

def gr_add_record(date, item, amount, need, cp, category):
    global record_df
    if not current_user:
        return "⚠️ 請先登入！", None
    new_row = {
        "日期": date,  # 直接存字串
        "帳號": current_user,
        "品項": item,
        "金額": amount,
        "需求": need,
        "CP值": cp,
        "分類": category
    }
    record_df = pd.concat([record_df, pd.DataFrame([new_row])], ignore_index=True)
    save_records(record_df)
    return "✅ 新增成功！", gr_show_records()

def gr_show_records():
    if not current_user:
        return pd.DataFrame()
    df = load_records()
    df = df[df['帳號']==current_user].copy()
    if df.empty:
        return pd.DataFrame()
    df['金額'] = pd.to_numeric(df['金額'], errors='coerce').fillna(0)
    return df

def gr_delete_record(idx):
    global record_df
    if not current_user:
        return "⚠️ 請先登入！", None
    if idx < 0 or idx >= len(record_df):
        return "⚠️ 索引錯誤！", gr_show_records()
    # 只刪當前使用者的資料
    record_df = record_df.drop(record_df[(record_df['帳號']==current_user) & (record_df.index==idx)].index)
    record_df.reset_index(drop=True, inplace=True)
    save_records(record_df)
    return "✅ 刪除成功！", gr_show_records()

with gr.Blocks() as demo:
    gr.Markdown("## 💜 線上記帳系統 💜")

    with gr.Tab("註冊"):
        user_reg = gr.Textbox(label="帳號")
        pwd_reg = gr.Textbox(label="密碼", type="password")
        btn_reg = gr.Button("註冊")
        out_reg = gr.Textbox(label="結果")
        btn_reg.click(fn=gr_register, inputs=[user_reg, pwd_reg], outputs=out_reg)

    with gr.Tab("登入"):
        user_login = gr.Textbox(label="帳號")
        pwd_login = gr.Textbox(label="密碼", type="password")
        btn_login = gr.Button("登入")
        out_login = gr.Textbox(label="結果")
        btn_login.click(fn=gr_login, inputs=[user_login, pwd_login], outputs=out_login)

    with gr.Tab("記帳"):
        date_in = gr.Textbox(label="日期 (YYYY-MM-DD)", value=datetime.today().strftime("%Y-%m-%d"))
        item_in = gr.Textbox(label="品項")
        amount_in = gr.Number(label="金額", value=0)
        need_in = gr.Radio(choices=['需要','想要'], label="需求", value='需要')
        cp_in = gr.Slider(1, 100, 50, label="CP值")
        cat_in = gr.Dropdown(choices=categories, label="分類", value=categories[0])
        btn_add = gr.Button("新增支出")
        out_add = gr.Textbox(label="結果")
        table = gr.Dataframe(label="支出紀錄", interactive=False)

        btn_add.click(fn=gr_add_record, inputs=[date_in, item_in, amount_in, need_in, cp_in, cat_in], outputs=[out_add, table])

    with gr.Tab("刪除紀錄"):
        idx_in = gr.Number(label="刪除索引 (從0開始)", value=0)
        btn_del = gr.Button("刪除")
        out_del = gr.Textbox(label="結果")
        table_del = gr.Dataframe(label="支出紀錄", interactive=False)

        btn_del.click(fn=gr_delete_record, inputs=idx_in, outputs=[out_del, table_del])

# 啟動 Gradio
demo.launch(share=True)